In [1]:
import urllib.request
#if the arg is empty in ProxyHandler, urllib will find itself your proxy config.
proxy_support = urllib.request.ProxyHandler({})
opener = urllib.request.build_opener(proxy_support)
urllib.request.install_opener(opener)



In [2]:
# import requests
# # make sure ES is up and running
# # for some reason, the root level endpoint of 9200 port is blocked. it's not a problem to access _search and other api endpoint though
# res = requests.get('http://localhost:9200')
# print(res.content)

In [3]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [4]:
#index some test data
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})

{'_index': 'test-index',
 '_type': 'test',
 '_id': '1',
 '_version': 3,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 1}

In [5]:
#delete test data and try with something more interesting
es.delete(index='test-index', doc_type='test', id=1)

{'_index': 'test-index',
 '_type': 'test',
 '_id': '1',
 '_version': 4,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 5,
 '_primary_term': 1}

In [6]:
es.index(index='sw', doc_type='people', id=1, body={
    "name": "Luke Skywalker",
    "height": "172",
    "mass": "77",
    "hair_color": "blond",
    "birth_year": "19BBY",
    "gender": "male",
})

{'_index': 'sw',
 '_type': 'people',
 '_id': '1',
 '_version': 5,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 108,
 '_primary_term': 1}

In [7]:
#let's iterate over swapi people documents and index them
import requests
import json
# r = requests.get('http://localhost:9200') 
status_code = 200
#print(r)
i = 1
while status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    status_code = r.status_code
    # print(status_code)
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
    
print(i)

18


In [ ]:
#let's iterate over swapi people documents and index them
import json
# r = requests.get('http://localhost:9200') 
status_code = 200
#print(r)
i = 18
while status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    status_code = r.status_code
    # print(status_code)
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
    
print(i)

In [ ]:
es.get(index='sw', doc_type='people', id=65)

In [ ]:
#let's find darth vader
es.search(index="sw", body={"query": {"match" : { "name" : "Darth Vader" }}})

In [ ]:
#let's see if we find anyone with a name or surname starting with lu
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

In [ ]:
#let's try a fuzzy query
q = es.search(index="sw", body={
    "query": {
       "fuzzy" : { "name" : {
                "value": "Vaded",
                "boost": 1.0,
                "fuzziness": 2,
                "prefix_length": 0,
                "max_expansions": 20
                } 
           }
        }
    }
)

In [ ]:
q